<a href="https://colab.research.google.com/github/NavyaNaveen08/Navya_SOC/blob/main/Web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install flask flask-ngrok joblib pandas numpy

In [6]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import joblib
import numpy as np

# Load trained model
model = joblib.load("optimized_timing_prediction_model.pkl")

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when running the app

@app.route("/")
def home():
    return "TimeSynth AI - Logic Depth Prediction API is running!"

@app.route("/predict", methods=["POST"])
def predict_logic_depth():
    try:
        # Get JSON data from request
        data = request.get_json()
        features = np.array([[data["fan_in"], data["fan_out"], data["num_gates"],
                              data["num_logic_levels"], data["path_delay"],
                              data["FOC"], data["SGPA"]]])

        # Make prediction
        prediction = model.predict(features)

        return jsonify({"predicted_logic_depth": float(prediction[0])})

    except Exception as e:
        return jsonify({"error": str(e)})

# Run Flask app
if __name__ == "__main__":
    app.run()

FileNotFoundError: [Errno 2] No such file or directory: 'optimized_timing_prediction_model.pkl'

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import joblib

# Define the path where the model is stored in Drive
drive_model_path = "/content/drive/My Drive/optimized_timing_prediction_model.pkl"

# Load the model
model = joblib.load(drive_model_path)

print("✅ Trained model loaded successfully!")


✅ Trained model loaded successfully!


In [10]:
import numpy as np

# Example input (adjust as per your feature set)
sample_input = np.array([[5, 10, 50, 8, 1.2]])  # [fan_in, fan_out, num_gates, ...]

# Make a prediction
prediction = model.predict(sample_input)

print(f"Predicted Logic Depth: {prediction[0]:.2f}")


Predicted Logic Depth: 13.63


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


In [12]:
%%writefile app.py
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import joblib
import numpy as np

# Load trained model from Drive
model = joblib.load("/content/drive/My Drive/optimized_timing_prediction_model.pkl")

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Expose Flask app via ngrok

@app.route("/")
def home():
    return "TimeSynth AI - Logic Depth Prediction API is running!"

@app.route("/predict", methods=["POST"])
def predict_logic_depth():
    try:
        data = request.get_json()
        features = np.array([[data["fan_in"], data["fan_out"], data["num_gates"],
                              data["num_logic_levels"], data["path_delay"],
                              data["FOC"], data["SGPA"]]])

        prediction = model.predict(features)
        return jsonify({"predicted_logic_depth": float(prediction[0])})

    except Exception as e:
        return jsonify({"error": str(e)})

# Run Flask app
if __name__ == "__main__":
    app.run()

Writing app.py


In [15]:
!python3 app.py

 * Serving Flask app 'app'
 * Debug mode: off
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
Usage of ngrok requires a verified account and authtoken.

Sign up for an account: https://dashboard.ngrok.com/signup
Install your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken

ERR_NGROK_4018

Exception in thread Thread-1:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.

In [14]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
added 22 packages in 3s
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼

In [16]:
from flask import Flask, render_template_string, request
import joblib
import numpy as np
import threading
import subprocess

# Load trained model from Google Drive
model = joblib.load("/content/drive/My Drive/optimized_timing_prediction_model.pkl")

# Initialize Flask app
app = Flask(__name__)

# HTML Page
html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>TimeSynth AI - Logic Depth Predictor</title>
</head>
<body style="text-align: center; font-family: Arial, sans-serif;">
    <h2>TimeSynth AI - Logic Depth Predictor</h2>
    <form method="post">
        <label>Fan In: <input type="number" name="fan_in" required></label><br><br>
        <label>Fan Out: <input type="number" name="fan_out" required></label><br><br>
        <label>Number of Gates: <input type="number" name="num_gates" required></label><br><br>
        <label>Number of Logic Levels: <input type="number" name="num_logic_levels" required></label><br><br>
        <label>Path Delay: <input type="number" name="path_delay" required></label><br><br>
        <label>FOC: <input type="number" name="FOC" required></label><br><br>
        <label>SGPA: <input type="number" name="SGPA" required></label><br><br>
        <button type="submit">Predict</button>
    </form>

    {% if prediction %}
    <h3>Predicted Logic Depth: {{ prediction }}</h3>
    {% endif %}
</body>
</html>
"""

@app.route("/", methods=["GET", "POST"])
def index():
    prediction = None
    if request.method == "POST":
        # Get input values from form
        features = np.array([[float(request.form["fan_in"]), float(request.form["fan_out"]),
                              float(request.form["num_gates"]), float(request.form["num_logic_levels"]),
                              float(request.form["path_delay"]), float(request.form["FOC"]),
                              float(request.form["SGPA"])]])

        # Predict logic depth
        prediction = model.predict(features)[0]

    return render_template_string(html_template, prediction=prediction)

# Function to start LocalTunnel
def start_localtunnel():
    subprocess.run(["lt", "--port", "5000", "--subdomain", "timesynth-ai"], check=True)

# Start Flask server in a separate thread
def run_flask():
    app.run(port=5000, debug=False, use_reloader=False)

# Run Flask and expose it via LocalTunnel
threading.Thread(target=run_flask).start()
threading.Thread(target=start_localtunnel).start()


 * Serving Flask app '__main__'
 * Debug mode: off


In [17]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://0mgr53a1i58-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [18]:
!cat /content/app.py

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import joblib
import numpy as np

# Load trained model from Drive
model = joblib.load("/content/drive/My Drive/optimized_timing_prediction_model.pkl")

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Expose Flask app via ngrok

@app.route("/")
def home():
    return "TimeSynth AI - Logic Depth Prediction API is running!"

@app.route("/predict", methods=["POST"])
def predict_logic_depth():
    try:
        data = request.get_json()
        features = np.array([[data["fan_in"], data["fan_out"], data["num_gates"],
                              data["num_logic_levels"], data["path_delay"],
                              data["FOC"], data["SGPA"]]])
        
        prediction = model.predict(features)
        return jsonify({"predicted_logic_depth": float(prediction[0])})

    except Exception as e:
        return jsonify({"error": str(e)})

# Run Flask app
if __name__ == "__main__":
    app

In [1]:
from flask import Flask, render_template_string, request, jsonify
import joblib
import numpy as np
import threading

# Load trained model from Google Drive
model = joblib.load("/content/drive/My Drive/optimized_timing_prediction_model.pkl")

# Initialize Flask app
app = Flask(__name__)

# Basic HTML Website
html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>TimeSynth AI - Logic Depth Predictor</title>
</head>
<body style="text-align: center; font-family: Arial, sans-serif;">
    <h2>TimeSynth AI - Logic Depth Predictor</h2>
    <form method="post">
        <label>Fan In: <input type="number" name="fan_in" required></label><br><br>
        <label>Fan Out: <input type="number" name="fan_out" required></label><br><br>
        <label>Number of Gates: <input type="number" name="num_gates" required></label><br><br>
        <label>Number of Logic Levels: <input type="number" name="num_logic_levels" required></label><br><br>
        <label>Path Delay: <input type="number" name="path_delay" required></label><br><br>
        <label>FOC: <input type="number" name="FOC" required></label><br><br>
        <label>SGPA: <input type="number" name="SGPA" required></label><br><br>
        <button type="submit">Predict</button>
    </form>

    {% if prediction %}
    <h3>Predicted Logic Depth: {{ prediction }}</h3>
    {% endif %}
</body>
</html>
"""

@app.route("/", methods=["GET", "POST"])
def index():
    prediction = None
    if request.method == "POST":
        try:
            # Get user input
            features = np.array([[float(request.form["fan_in"]), float(request.form["fan_out"]),
                                  float(request.form["num_gates"]), float(request.form["num_logic_levels"]),
                                  float(request.form["path_delay"]), float(request.form["FOC"]),
                                  float(request.form["SGPA"])]])

            # Make prediction
            prediction = model.predict(features)[0]

        except Exception as e:
            prediction = f"Error: {str(e)}"

    return render_template_string(html_template, prediction=prediction)

# Function to run Flask in the background
def run_flask():
    app.run(port=5000)

# Start Flask in a separate thread
threading.Thread(target=run_flask).start()


 * Serving Flask app '__main__'
 * Debug mode: off


In [2]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))


https://0mgr53a1i58-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [3]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

In [1]:
import gradio as gr
import joblib
import numpy as np

# Load trained model from Google Drive
model = joblib.load("/content/drive/My Drive/optimized_timing_prediction_model.pkl")

# Define prediction function
def predict_logic_depth(fan_in, fan_out, num_gates, num_logic_levels, path_delay, FOC, SGPA):
    features = np.array([[fan_in, fan_out, num_gates, num_logic_levels, path_delay, FOC, SGPA]])
    prediction = model.predict(features)
    return f"Predicted Logic Depth: {prediction[0]:.2f}"

# Create Gradio UI
interface = gr.Interface(
    fn=predict_logic_depth,
    inputs=[
        gr.Number(label="Fan In"),
        gr.Number(label="Fan Out"),
        gr.Number(label="Number of Gates"),
        gr.Number(label="Number of Logic Levels"),
        gr.Number(label="Path Delay"),
        gr.Number(label="FOC"),
        gr.Number(label="SGPA")
    ],
    outputs=gr.Textbox(label="Prediction"),
    title="TimeSynth AI - Logic Depth Predictor",
    description="Enter RTL parameters and get a predicted logic depth."
)

# Launch the web app
interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5620a963175fe57a59.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
